In [1]:
import lancedb
import polars as pl

## Import and load model
from langchain_community.embeddings import HuggingFaceEmbeddings
model = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

## Connect to the database we've already created
db = lancedb.connect("../temp_data/gm_lancedb/")

## Load the table we've already created
table = db.open_table("gm_table")

In [2]:
%%time
## Asking a query before building ANN index:

## Embed a query
query_text = "What are the biggest benefits of working for GM?"
query = model.embed_query(query_text)

## Perform similarity search on database using KNN
result = table.search(query).limit(15).to_polars()

## Display the results
pl.Config.set_tbl_rows(15) ## Display up to 15 rows
pl.Config.set_fmt_str_lengths(3000) ## Display up to 3000 characters per column
result[["reddit_subreddit","reddit_text","text_chunk"]]

CPU times: user 128 ms, sys: 124 ms, total: 252 ms
Wall time: 322 ms


reddit_subreddit,reddit_text,text_chunk
str,str,str
"""GeneralMotors""","""If you work at GM most of the benefits that make this job worthwhile come from unions demanding them""","""If you work at GM most of the benefits that make this job worthwhile come from unions demanding them"""
"""GeneralMotors""","""In general GM has pretty good benefits, a good work life balance, and is on the lower side for base pay. You will have to figure out for yourself how much things like different health care plans, work life balance, and time off are worth. For me the reason I chose to look for a new position and leave GM was the lack of learning on the job and lack of mentorship inside my team at the time. But that is entirely team based. If I was on a different team there is a likelihood I would not have looked and left for a new job. Ultimately I'm happy for the way my experience at GM shaped my career priorities and for the time I was there. I'm also glad I got out when I did as I was not in a good position to grow like I wanted to.""","""In general GM has pretty good benefits, a good work life balance, and is on the lower side for base pay. You will have to figure out for yourself how much things like different health care plans, work life balance, and time off are worth."""
"""GeneralMotors""","""As a former employee who has moved on from the company I can say the benefits at GM are pretty good and are a good reason to stay. And the company is usually (but not always) good at making cost of living increases. With good benefits and salary increases its pretty easy to get comfortable at GM.""","""As a former employee who has moved on from the company I can say the benefits at GM are pretty good and are a good reason to stay. And the company is usually (but not always) good at making cost of living increases. With good benefits and salary increases its pretty easy to get comfortable at GM."""
"""GeneralMotors""","""I do agree that GM's overall benefits are superior. A couple things to note is that the other company is offering $2k more in tuition assistance than GM if I do grad school, which is what I would pursue (if I need to) since I'm almost done with my undergrad. GM's vacation policy and paternity leave is better, but do fathers get the 12 weeks too or is that for mothers? The other company has the same matching policy except they don't give you 4% of your salary for free even if you don't personally contribute. The other company doesn't have a vesting period while GM has a 3 year vesting period. The main things the other company brings to the table are additional bonuses such as a $10k sign on bonus as well as $15k in stocks and a sizeable lump sump for relocation, on top of around $7k more in base pay. They also have a similar 10% bonus system where they very often hit their 200% target compared to GM.""","""I do agree that GM's overall benefits are superior."""
"""GeneralMotors""","""What makes GM better? Better work life balance?""","""What makes GM better? Better work life balance?"""
"""GeneralMotors""","""GM has one of the best benefits out there. Look at the 1500 for life steps program. GM will give you 1500 just to get your physical done. The GM culture and holidays we get beats most companies. Did you know we get a day off just to vote? Primary and secondary elections? The shutdown for Christmas is a week. Free vacation. And did you know GM gives 12 weeks paid paternity leave? All of those summed up will be close or even more than the other company. Also GM gives you 4% on 401k and match of what you put in. It is 12% max i think. That is also huge. Sharing these benefits are not secret and is publix info if you dig it up. So what is the other company bringing to the table? Also GM will pay for your school if you were to obtain a masters degree.""","""GM has one of the best benefits out there. Look at the 1500 for life steps program. GM will give you 1500 just to get your physical done. The GM culture and holidays we get bea

In [4]:
%%time
## Build the ANN Index
## https://lancedb.github.io/lancedb/ann_indexes/#creating-an-ivf_pq-index
table.create_index(
    num_partitions=2,
    num_sub_vectors=96,
    accelerator="mps" ## to use Apple Metal
)
## It is safe to ignore the UserWarning:...

  8%|██████████▉                                                                                                                              | 4/50 [00:00<00:01, 27.00it/s]


0it [00:00, ?it/s]

/Users/marcos/miniconda3/envs/aware/lib/python3.11/site-packages/lance/torch/data.py:67: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:212.)
  tensor = torch.from_numpy(arr.to_numpy(zero_copy_only=False))


CPU times: user 2min 29s, sys: 6.78 s, total: 2min 36s
Wall time: 14.2 s


In [6]:
%%time
## Repeat the query using the ANN index
## https://lancedb.github.io/lancedb/ann_indexes/#querying-an-ann-index

## Embed a query
query_text = "What are the biggest benefits of working for GM?"
query = model.embed_query(query_text)

## Perform similarity search on database using KNN
result = table.search(query) \
    .limit(15) \
    .nprobes(20) \
    .refine_factor(10) \
    .to_polars()

## Display the results
pl.Config.set_tbl_rows(15) ## Display up to 15 rows
pl.Config.set_fmt_str_lengths(3000) ## Display up to 3000 characters per column
result[["reddit_subreddit","reddit_text","text_chunk"]]

CPU times: user 38 ms, sys: 15.4 ms, total: 53.4 ms
Wall time: 86.9 ms


reddit_subreddit,reddit_text,text_chunk
str,str,str
"""GeneralMotors""","""If you work at GM most of the benefits that make this job worthwhile come from unions demanding them""","""If you work at GM most of the benefits that make this job worthwhile come from unions demanding them"""
"""GeneralMotors""","""In general GM has pretty good benefits, a good work life balance, and is on the lower side for base pay. You will have to figure out for yourself how much things like different health care plans, work life balance, and time off are worth. For me the reason I chose to look for a new position and leave GM was the lack of learning on the job and lack of mentorship inside my team at the time. But that is entirely team based. If I was on a different team there is a likelihood I would not have looked and left for a new job. Ultimately I'm happy for the way my experience at GM shaped my career priorities and for the time I was there. I'm also glad I got out when I did as I was not in a good position to grow like I wanted to.""","""In general GM has pretty good benefits, a good work life balance, and is on the lower side for base pay. You will have to figure out for yourself how much things like different health care plans, work life balance, and time off are worth."""
"""GeneralMotors""","""As a former employee who has moved on from the company I can say the benefits at GM are pretty good and are a good reason to stay. And the company is usually (but not always) good at making cost of living increases. With good benefits and salary increases its pretty easy to get comfortable at GM.""","""As a former employee who has moved on from the company I can say the benefits at GM are pretty good and are a good reason to stay. And the company is usually (but not always) good at making cost of living increases. With good benefits and salary increases its pretty easy to get comfortable at GM."""
"""GeneralMotors""","""I do agree that GM's overall benefits are superior. A couple things to note is that the other company is offering $2k more in tuition assistance than GM if I do grad school, which is what I would pursue (if I need to) since I'm almost done with my undergrad. GM's vacation policy and paternity leave is better, but do fathers get the 12 weeks too or is that for mothers? The other company has the same matching policy except they don't give you 4% of your salary for free even if you don't personally contribute. The other company doesn't have a vesting period while GM has a 3 year vesting period. The main things the other company brings to the table are additional bonuses such as a $10k sign on bonus as well as $15k in stocks and a sizeable lump sump for relocation, on top of around $7k more in base pay. They also have a similar 10% bonus system where they very often hit their 200% target compared to GM.""","""I do agree that GM's overall benefits are superior."""
"""GeneralMotors""","""What makes GM better? Better work life balance?""","""What makes GM better? Better work life balance?"""
"""GeneralMotors""","""GM has one of the best benefits out there. Look at the 1500 for life steps program. GM will give you 1500 just to get your physical done. The GM culture and holidays we get beats most companies. Did you know we get a day off just to vote? Primary and secondary elections? The shutdown for Christmas is a week. Free vacation. And did you know GM gives 12 weeks paid paternity leave? All of those summed up will be close or even more than the other company. Also GM gives you 4% on 401k and match of what you put in. It is 12% max i think. That is also huge. Sharing these benefits are not secret and is publix info if you dig it up. So what is the other company bringing to the table? Also GM will pay for your school if you were to obtain a masters degree.""","""GM has one of the best benefits out there. Look at the 1500 for life steps program. GM will give you 1500 just to get your physical done. The GM culture and holidays we get bea

In [7]:
## Write a function to play with queries and ANN parameters
import time

def make_query(query_text, 
               num_results=10, 
               probes=20, 
               refine_factor=10):
    
    start_time = time.time()
    ## Embed query
    query = model.embed_query(query_text)

    ## Perform search
    result = table.search(query).limit(num_results)\
        .nprobes(20)\
        .refine_factor(10)\
        .to_polars()
    
    end_time = time.time()
    print(f"Time elapsed this query: {(end_time - start_time):.4f} seconds")
    return result

In [8]:
queries = ["How many days off do you get every year working for GM?", 
           "What are the best parts of working for a GM?", 
           "What are the biggest red flags in a GM interview?", 
           "Do GM workers have to handle customer often?", 
           "Do people who work at GM stay for a long time?"]

pl.Config.set_fmt_str_lengths(3000)
results = []
for query in queries:
    result = make_query(query)
    results.append([query,result])
    #print(f"Query: {query}")
    #print(result[["aware_post_type","reddit_subreddit", "reddit_text", "text_chunk", "_distance"]])

Time elapsed this query: 0.2457 seconds
Time elapsed this query: 0.0633 seconds
Time elapsed this query: 0.0241 seconds
Time elapsed this query: 0.1837 seconds
Time elapsed this query: 0.1885 seconds


In [9]:
print(f"Query: {results[0][0]}")
results[0][1][["aware_post_type","reddit_subreddit", "reddit_text", "text_chunk", "_distance"]]

Query: How many days off do you get every year working for GM?


aware_post_type,reddit_subreddit,reddit_text,text_chunk,_distance
str,str,str,str,f32
"""comment""","""GeneralMotors""","""GM has 16 holidays and 3 weeks vacation and unlimited sick. You'll get plenty of time off. After two years you go to four weeks vacation. That's the only thing I'll miss with my new job. That week between Christmas and New Year is amazing.""","""GM has 16 holidays and 3 weeks vacation and unlimited sick. You'll get plenty of time off. After two years you go to four weeks vacation. That's the only thing I'll miss with my new job. That week between Christmas and New Year is amazing.""",0.148321
"""comment""","""GeneralMotors""","""Hey, I'm a PM and came from another OEM where I was also a PM. The fact that they are hiring you through VSP and trying to reduce headcount makes me think you are probably safe. There's always the chance to be laid off regardless, but it seems they're over the hill so to speak on it for now. I don't know if they will change the hybrid requirement, but they certainly could. I was hired fully remote but they still tried to tell me I had to come in as part of RTO. Thankfully I had receipts and they honored it but they totally will bait and switch you on this. I'd say there's zero chance of changing from hybrid to fully remote. Work life balance is the best part of GM imo. They give you a ton of holidays off, starting vacation at 15 days which goes up to 20 after 2 years, great leave of absences if you need to care for yourself or another, etc. I rarely work overtime and never on weekends. The people I've worked with have been very considerate of my time and life outside of work. I feel whole working at GM. You didn't ask this but in my experience being a PM here is not a true PM role. I think this depends on your area, but where I'm at they are really struggling to embrace agility and no one on my team has prior PM experience. We are handed initiatives from leadership and told to make them work, so there isn't a lot of opportunity to innovate or use customer feedback/insights to drive change. Products are very messy. It's disappointing to not have any real control over my product, but this also means more work life balance for me because a significant part of the PM job is lifted off my shoulders. So take that how you will. I've heard it's better in some areas.""","""Work life balance is the best part of GM imo. They give you a ton of holidays off, starting vacation at 15 days which goes up to 20 after 2 years, great leave of absences if you need to care for yourself or another, etc. I rarely work overtime and never on weekends. The people I've worked with have been very considerate of my time and life outside of work. I feel whole working at GM.""",0.172856
"""submission""","""GeneralMotors""","""365 days a year. Needless to say, I support the workers. I wouldn’t buy a gm car until the workers are satisfied.""","""365 days a year. Needless to say, I support the workers. I wouldn’t buy a gm car until the workers are satisfied.""",0.17774
"""comment""","""GeneralMotors""","""This is news to me. I’ve worked for GM for almost a decade and have never put any sort of time off into a formal system.""","""This is news to me. I’ve worked for GM for almost a decade and have never put any sort of time off into a formal system.""",0.182406
"""comment""","""GeneralMotors""","""Yeah, 3 years. The PTO is better than most companies. It took me over 7 years to get 20 days off at my former company and they only gave about 9-10 holidays. I have more vacay + holiday time off now at GM at just the 2 year mark than I ever did.""","""Yeah, 3 years. The PTO is better than most companies. It took me over 7 years to get 20 days off at my former company and they only gave about 9-10 holidays. I have more vacay + holiday time off now at GM at just the 2 year mark than I ever did.""",0.190434
"""comment""","""GeneralMotors""","""I get 3 weeks PTO (includes sick time) and 9 paid holidays. I was up to five weeks v

In [10]:
print(f"Query: {results[1][0]}")
results[1][1][["aware_post_type","reddit_subreddit", "reddit_text", "text_chunk", "_distance"]]

Query: What are the best parts of working for a GM?


aware_post_type,reddit_subreddit,reddit_text,text_chunk,_distance
str,str,str,str,f32
"""comment""","""GeneralMotors""","""Hey, I'm a PM and came from another OEM where I was also a PM. The fact that they are hiring you through VSP and trying to reduce headcount makes me think you are probably safe. There's always the chance to be laid off regardless, but it seems they're over the hill so to speak on it for now. I don't know if they will change the hybrid requirement, but they certainly could. I was hired fully remote but they still tried to tell me I had to come in as part of RTO. Thankfully I had receipts and they honored it but they totally will bait and switch you on this. I'd say there's zero chance of changing from hybrid to fully remote. Work life balance is the best part of GM imo. They give you a ton of holidays off, starting vacation at 15 days which goes up to 20 after 2 years, great leave of absences if you need to care for yourself or another, etc. I rarely work overtime and never on weekends. The people I've worked with have been very considerate of my time and life outside of work. I feel whole working at GM. You didn't ask this but in my experience being a PM here is not a true PM role. I think this depends on your area, but where I'm at they are really struggling to embrace agility and no one on my team has prior PM experience. We are handed initiatives from leadership and told to make them work, so there isn't a lot of opportunity to innovate or use customer feedback/insights to drive change. Products are very messy. It's disappointing to not have any real control over my product, but this also means more work life balance for me because a significant part of the PM job is lifted off my shoulders. So take that how you will. I've heard it's better in some areas.""","""Work life balance is the best part of GM imo. They give you a ton of holidays off, starting vacation at 15 days which goes up to 20 after 2 years, great leave of absences if you need to care for yourself or another, etc. I rarely work overtime and never on weekends. The people I've worked with have been very considerate of my time and life outside of work. I feel whole working at GM.""",0.186306
"""comment""","""GeneralMotors""","""Same position, but higher salary. Was a bit smarter and knew how to negotiate my salary better, but also negotiated a restoration of my vacation days (although my seniority tracking for additional days is reset to 0) I came back for 2 reasons - Biggest was that I was extremely disappointed in my new job. Didn't line up to expectations, despite getting paid a lot more. Secondary reason - despite what you read here, GM is actually a decent place to work *if you find the right division and right leadership.* I joined back up with a director I worked closely with, a team that I had a great rep with, and a fascinating growth division.""","""Secondary reason - despite what you read here, GM is actually a decent place to work *if you find the right division and right leadership.* I joined back up with a director I worked closely with, a team that I had a great rep with, and a fascinating growth division.""",0.192753
"""comment""","""GeneralMotors""","""Great answer, and thank you. Getting paid more / trying a new field would be reasons to leave on my end. Agreed it's not a bad place to work with the right leadership, lots of big perks to GM!""","""Great answer, and thank you. Getting paid more / trying a new field would be reasons to leave on my end. Agreed it's not a bad place to work with the right leadership, lots of big perks to GM!""",0.195453
"""comment""","""GeneralMotors""","""I don't particularly care where someone worked prior to GM, unless there are integrity concerns. As for any role in GM at any level, to convince me someone is on the ""right track"" would be to * Train and acclimate well to their role and not be afraid to ask questions. * Either know what you are doing, or show true effort in attempting to know 

In [11]:
print(f"Query: {results[2][0]}")
results[2][1][["aware_post_type","reddit_subreddit", "reddit_text", "text_chunk", "_distance"]]

Query: What are the biggest red flags in a GM interview?


aware_post_type,reddit_subreddit,reddit_text,text_chunk,_distance
str,str,str,str,f32
"""comment""","""GeneralMotors""","""Interview immediately for new roles, if you smell something fishy within GM, there’s definitely something fishy going on""","""Interview immediately for new roles, if you smell something fishy within GM, there’s definitely something fishy going on""",0.179821
"""comment""","""GeneralMotors""","""I don't particularly care where someone worked prior to GM, unless there are integrity concerns. As for any role in GM at any level, to convince me someone is on the ""right track"" would be to * Train and acclimate well to their role and not be afraid to ask questions. * Either know what you are doing, or show true effort in attempting to know what you are doing. * Follow reasonable deadlines/demand and push back when unreasonable deadlines/demand are assigned to them. * Make data-driven decisions. A trendy example would be ""don't redesign infotainment without both proving it is attainable and achieves a higher customer satisfaction"" (elimination of CarPlay). I really think if data on this, in particular customer survey data, was sufficiently provided, there would be way less backlash. * Launching good products is more important that new products. As much as I'd love to operate on an infinite budget and get experimental, the reality is, if customers don't want the new product, then it is a bad product. ""Good"" and ""bad"" are subjective, but revenue isn't. * I think making better efforts to effectively train and retain devs is more important than ""hiring a totally different caliber of devs"". The best developer in the world won't do a good job if they don't understand the task. Pushing out a knowledge base may cause more problems than it will fix. Sometimes it is better to demolish and rebuild than renovate, but life is often not that black and white. * Personally, more transparency on the long term plans or roadmaps is always appreciated. I won't know if I should or shouldn't be confident if I don't even see the big picture. * I don't work in software. However, a big annoyance I have had is it seems there is not a lot of commonization in tools/systems. I'm willing to admit because it is not my area of expertise, I may be wrong, and every team using their own unique systems could be the best. But from my limited view, it makes it harder to collaborate with different teams. * Re-org'ing can be good. Sometimes managers are overseeing employee's whose work they don't understand at all. Teams should make sense. Ultimately, though, implementing rules such as ""every manager must supervise at least X employees"" and ""flattening the org"" just reads as ""we overpayed a consulting group to tell us how to organize, based exactly on what they told a similar company to do"". Ask your employees if what they are doing makes sense to be in the team they are in, or if they would be better suited doing the same work for another team. I'm sure this list is not exhaustive and could be better written. But these are my impulsive thoughts. If I think more on it, maybe some opinions could be changed. TL;DR: I don't care if new workers are from Apple or anywhere else. Make decisions that are popular, or prove why unpopular decisions are better. No one is better than anyone else. If you are managing a team remotely, then your workers can work remotely.""","""I don't particularly care where someone worked prior to GM, unless there are integrity concerns. As for any role in GM at any level, to convince me someone is on the ""right track"" would be to""",0.222112
"""comment""","""GeneralMotors""","""A GM hiring manager here: I always aim to interview at least 3 people per role. If no one shines, we keep the job req open and keep going. Good luck!!""","""A GM hiring manager here: I always aim to interview at least 3 people per role. If no one shines, we keep the job req open and keep going. Good luck!!""",0.243239
"""comment""","""Gener

In [12]:
print(f"Query: {results[3][0]}")
results[3][1][["aware_post_type","reddit_subreddit", "reddit_text", "text_chunk", "_distance"]]

Query: Do GM workers have to handle customer often?


aware_post_type,reddit_subreddit,reddit_text,text_chunk,_distance
str,str,str,str,f32
"""comment""","""GeneralMotors""","""Do you work for GM?""","""Do you work for GM?""",0.202784
"""comment""","""GeneralMotors""","""Do you work for GM?""","""Do you work for GM?""",0.202784
"""comment""","""GeneralMotors""","""Yes, I work for GM. I have never been ""mandated"" to work OT. I am the type that gets the job done.""","""Yes, I work for GM. I have never been ""mandated"" to work OT. I am the type that gets the job done.""",0.203964
"""comment""","""GeneralMotors""","""Understood. thank you! I am not a GM employee (was formerly). Figured I’d join it for any updates on the company itself or news. Would you have any input to as why a company like GM would cycle through so many employees within a 1-2 month timeframe? I assumed it was just some corporate background work and such but I’m not completely sure what the reasoning was i just know that they had let a tech go the week before I was given the boot and was told to get ready and was told (they were cleaning out the shop)… which seemed like a unlikely answer""","""Understood. thank you! I am not a GM employee (was formerly). Figured I’d join it for any updates on the company itself or news. Would you have any input to as why a company like GM would cycle through so many employees within a 1-2 month timeframe? I assumed it was just some corporate background work and such but I’m not completely sure what the reasoning was i just know that they had let a tech go the week before I was given the boot and was told to get ready and was told (they were cleaning out the""",0.210218
"""comment""","""GeneralMotors""","""I agree. Folks in manufacturing have to come in every day. GM is a manufacturing company. All roles support manufacturing.""","""I agree. Folks in manufacturing have to come in every day. GM is a manufacturing company. All roles support manufacturing.""",0.212837
"""comment""","""GeneralMotors""","""You are obviously not a GM employee. You’re obviously someone with a lot of time on your hands.""","""You are obviously not a GM employee. You’re obviously someone with a lot of time on your hands.""",0.220113
"""comment""","""GeneralMotors""","""I don’t doubt that, but there are a whole lot (the majority) of GM employees who aren’t IT.""","""I don’t doubt that, but there are a whole lot (the majority) of GM employees who aren’t IT.""",0.22021
"""comment""","""GeneralMotors""","""Do you work in Labor Relations for GM?""","""Do you work in Labor Relations for GM?""",0.221157
"""comment""","""GeneralMotors""","""Im a manufacturing engineer for a different company. Do gm engineers have to be there for production at all times? Our facility runs production most weekends but I still get to chill out.""","""Im a manufacturing engineer for a different company. Do gm engineers have to be there for production at all times? Our facility runs production most weekends but I still get to chill out.""",0.222607


In [13]:
print(f"Query: {results[4][0]}")
results[4][1][["aware_post_type","reddit_subreddit", "reddit_text", "text_chunk", "_distance"]]

Query: Do people who work at GM stay for a long time?


aware_post_type,reddit_subreddit,reddit_text,text_chunk,_distance
str,str,str,str,f32
"""comment""","""GeneralMotors""","""We are NOT talking about what’s happening. We are discussing: 1. Bringing this forward to the leadership when an employee decides to leave during maternity. You were arguing that this likely doesn’t happen. I brought it up to you that if I was the manager, I’d certainly flag it. Burned a bridge or not, it is up case to case. 2. The op was simply asking about if the scenario would burn a bridge and here you are ranting about fictitious scenario that may be far from reality of the situation. After all anyone’s employment at gm is at will. You may stay or leave as you please. Anyone that quits leaves a hole.""","""After all anyone’s employment at gm is at will. You may stay or leave as you please. Anyone that quits leaves a hole.""",0.166175
"""comment""","""GeneralMotors""","""Why have you stayed at GM for so long? Have you considered moving to a different company?""","""Why have you stayed at GM for so long? Have you considered moving to a different company?""",0.168869
"""comment""","""GeneralMotors""","""I think for most of the people who are going to leave its either keeping remote or at least they get paid more than at GM""","""I think for most of the people who are going to leave its either keeping remote or at least they get paid more than at GM""",0.174666
"""submission""","""GeneralMotors""","""Do you guys recommend to work at GM as a full time employee?""","""Do you guys recommend to work at GM as a full time employee?""",0.174708
"""comment""","""GeneralMotors""","""You’ve clearly worked here normal but in my 10 years. I have been seeing so many people just coasting not doing much work sticking around in GM But recently the culture has shifted. Now you get the boot that’s a huge culture shift. The same exact person that has been working in my team for almost 6 years recently got the boot because it was very clear they were not performing and they weren’t performing for years but they didn’t feel the need to because there was no performance based firing back in the day""","""You’ve clearly worked here normal but in my 10 years. I have been seeing so many people just coasting not doing much work sticking around in GM""",0.177717
"""comment""","""GeneralMotors""","""Jump to another company. Repeat every 3 years or so for a while, then settle in for long term. Source: GM retiree that worked whole career at GM.""","""Jump to another company. Repeat every 3 years or so for a while, then settle in for long term. Source: GM retiree that worked whole career at GM.""",0.179182
"""comment""","""GeneralMotors""","""I would not recommend anyone do this for **any** company, not just GM. If you stay 10 years at one place, you're likely only being paid 50% of what you're actually worth on the open market. The only exception to this rule is if your place of employment offers exceptional benefits & WLB. Apart from that, you should always look to jump ship every 2-4 years.""","""I would not recommend anyone do this for **any** company, not just GM. If you stay 10 years at one place, you're likely only being paid 50% of what you're actually worth on the open market. The only exception to this rule is if your place of employment offers exceptional benefits & WLB. Apart from that, you should always look to jump ship every 2-4 years.""",0.18217
"""comment""","""GeneralMotors""","""I’m literally using my own first hand data from the year me and most of my friends hired in, so your statement is factually incorrect. Also “oh we had it worse” is such a stupid argument for pay. Everyone deserves more and just because one group is fucked over more doesn’t mean people aren’t being fucked over now too. Also, I did the math and wages would have had to rise by 37% since 2011 and 30% since 2015 JUST to keep up with inflation, not go above it, and again 22.5% of that was in the last 5 years. Without getting any promotions people hi